In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

!pwd

Mounted at /content/gdrive/


In [3]:
!pip install wandb -qqq
%cd 'path/to/scripts/here'

import os
import numpy as np
import random
import wandb as wandb
from wandb import init

import torch
from torchvision.transforms import Compose, ToTensor, Normalize, RandomRotation, ToPILImage

from networks import *
from datasets import *
from utils import *

     |████████████████████████████████| 2.1MB 8.0MB/s 
     |████████████████████████████████| 102kB 12.5MB/s 
     |████████████████████████████████| 174kB 34.7MB/s 
     |████████████████████████████████| 133kB 33.4MB/s 
     |████████████████████████████████| 71kB 10.9MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=914ccb988ace8ebce723869ae5edde47ab0822e62b6e9112f4d3b3d122d2429f
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=9691806db78199e9ba208d37cfcbbfaf2f7a3a7582eee62bdcad6c0872e4dffe
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pathtools subprocess32
/content/gdrive/MyDrive/IC_PHD/GTA/ACSE20-21/ACSE4


In [4]:
!wandb login
!wandb --version

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb, version 0.10.27


In [5]:
# Set training and testing/validate transforms for normalisation and data augmentation
train_transform = Compose([
    # ToPILImage(),
    # RandomRotation(10),
    # ToTensor(),
    Normalize(mean=[0.5064], std=[0.2493]), 
])

test_transform = Compose([
    Normalize(mean=[0.5064], std=[0.2493])
])

In [12]:
# Set training static parameters and hyperparameters
parameters = dict(
    nepochs=100,
    
    data_name="xray-data",                                                 # will search folder ./{data_name}/train and ./{data_name}/test
    target_code={"covid":0, "lung_opacity":1, "pneumonia":2, "normal":3},  # each key should be a subfolder with the training data ./{data_name}/train/{key}
    input_height=299,
    input_width=299,
    input_channels=1,
    
    batch_size=128,
    test_batch_size=64,
    learning_rate=1e-2,
    momentum=0.0,
    
    model_name="ResNet18Wrapper",                                           # will search model in networks.py, must be a subclass of nn.Module and take as input param "num_classes" referring to the number of classes for classification
    dataset="XRAYTensorDataset",                                            # will search dataset in datasets.py, must be a subclass of torch.utils.data.Dataset, inputs must be img_paths, targets and transform, output of __getitem__ must be an image sample (C, H, W) and its target
    criterion_name="CrossEntropyLoss",
    optimizer_name="SGD",
    
    valid_split=0.2,
    train_transform=train_transform,
    test_transform=test_transform,
    
    device=set_device("cuda"),
    )

Cuda installed! Running on GPU Tesla P100-PCIE-16GB!


In [13]:
# tell wandb to get started
with wandb.init(project="rsna-covid", config=parameters, entity="acse4-2021-gtas") as wb:  
    # Model saving settings
    save_model = True
    save_frequency = 5
    
    # access all HPs through wandb.config, so logging matches execution!
    config = wandb.config

    # Set seed
    set_seed(42)
    
    # Set up run with parameters
    model, criterion, optimizer, train_loader, valid_loader = set_up_train(config, train_transform=parameters["train_transform"], valid_transform=parameters["test_transform"])
  
    # # Let wandb watch the model and the criterion
    wandb.watch(model, criterion)
    
    # Training loop
    print("\n\nTraining started ...")
    for epoch in range(config.nepochs):
        train_loss, train_accuracy = train(model, optimizer, criterion, train_loader, config)
        validation_loss, validation_accuracy = validate(model, criterion, valid_loader, config)
        
        log = {"epoch": epoch, "train_loss":train_loss.item(), "train_accuracy": train_accuracy.item(), "valid_loss":validation_loss.item(), "valid_accuracy":validation_accuracy.item()}
        wandb.log(log)
        print(log)

        # Saving model
        if save_model and (epoch % save_frequency == 0):
            model_dir =  os.path.join(wb.dir, "%s_epoch%g.pth"%(wb.name, epoch))
            print("\n Saving model at %s \n"%(model_dir))
            torch.save(model.state_dict(), model_dir)

Reading image paths in ./xray-data/train


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


 nepochs                                       :                           100
 data_name                                     :                     xray-data
 target_code                                   :               {'covid': 0, 'lung_opacity': 1, 'pneumonia': 2, 'normal': 3}
 input_height                                  :                           299
 input_width                                   :                           299
 input_channels                                :                             1
 batch_size                                    :                           128
 test_batch_size                               :                            64
 learning_rate                                 :                          0.01
 momentum                                      :                           0.0
 model_name                                    :               ResNet18Wrapper
 dataset                                       :               XRAYTensorDataset
 crit

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
100% (127 of 127) |######################| Elapsed Time: 0:00:57 Time:  0:00:57
100% (64 of 64) |########################| Elapsed Time: 0:00:12 Time:  0:00:12


{'epoch': 0, 'train_loss': 0.9587695002555847, 'train_accuracy': 0.6092010883007668, 'valid_loss': 0.9134908318519592, 'valid_accuracy': 0.60375958446698}

 Saving model at /content/gdrive/My Drive/IC_PHD/GTA/ACSE20-21/ACSE4/wandb/run-20210421_163351-2ey7a5te/files/earthy-butterfly-6_epoch0.pth 



...

{'epoch': 7, 'train_loss': 0.31045785546302795, 'train_accuracy': 0.8877689834281474, 'valid_loss': 2.4541101455688477, 'valid_accuracy': 0.5923818946326985}


100% (127 of 127) |######################| Elapsed Time: 0:00:55 Time:  0:00:55


epoch,7
train_loss,0.31046
train_accuracy,0.88777
valid_loss,2.45411
valid_accuracy,0.59238
_runtime,619
_timestamp,1619023450
_step,7


epoch,▁▂▃▄▅▆▇█
train_loss,█▅▄▃▂▂▁▁
train_accuracy,▁▄▅▆▇▇██
valid_loss,▂▁▁▁▁▂▁█
valid_accuracy,▁▆▇██▆▇▁
_runtime,▁▂▃▄▅▆▇█
_timestamp,▁▂▃▄▅▆▇█
_step,▁▂▃▄▅▆▇█


KeyboardInterrupt: ignored